<a name="Exercise"></a>

### Exercises

#### CRUD operations

Create a new db name Todo and a new collection named "CRUD_exercise" and do the following:

**TODO 1**: Take the dict created in the TODO 4 in chapter I and save it in the collection "CRUD_exercise".

**TODO 2**: Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

**TODO 3**: Insert the dict created in the TODO 6 Chapter I in the example collection.

**TODO 4**: Get documents where authors key exist in the collection "CRUD_exercise".

**TODO 5**: Change the documents where x = 4 to x = 1.

**TODO 6**: Find documents where author is not_mike and set author as real_mike.

**TODO 7**: Delete documents where author is real_mike.

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['Todo']
collection = db['CRUD_exercise']
print("Database and Collection Created: ", db.name, collection.name)

**TODO 1**: Take the dict created in the TODO 4 in chapter I and save it in the collection "CRUD_exercise".

In [1]:
import sqlite3

conn = sqlite3.connect('/Users/valen/Desktop/SAS/BC.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM tomatch3")
rows = cursor.fetchall()

data = [{'id': row[0], 'rdm_float': row[1]} for row in rows]

conn.close()

In [2]:
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient('localhost', 27017)
db = client['Todo']
collection = db['CRUD_exercise']

# Insert data into MongoDB
collection.insert_many(data)

# Verify insertion
print(collection.name)

Data inserted into MongoDB collection: CRUD_exercise


**TODO 2**: Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

In [3]:
#Insert 3 documents with key = x and values = 1
docs = [{'x': 1}, {'x': 1}, {'x': 1}]
insert_result = collection.insert_many(docs)
print(f"Inserted document IDs: {insert_result.inserted_ids}")

Inserted document IDs: [ObjectId('6630ee93e01c5a30842e1678'), ObjectId('6630ee93e01c5a30842e1679'), ObjectId('6630ee93e01c5a30842e167a')]


In [4]:
#delete one of them
delete_result = collection.delete_one({'x': 1})
print(f"Deleted count: {delete_result.deleted_count}")

Deleted count: 1


MongoDB ne va pas prendre en compte le plus récent ou plus ancien des documents mais supprimer le premier qu'il rencontre ayant la caractéristique demandé, dans notre cas, nous avons trois documents "{'x': 1}", donc il va en supprimer un au hasard s'il n'y a pas plus de spécification

In [5]:
#increment the value of x to 4
update_result = collection.update_one({'x': 1}, {'$set': {'x': 4}})
print(f"Updated count: {update_result.modified_count}")

Updated count: 1


**TODO 3**: Insert the dict created in the TODO 6 Chapter I in the example collection.

In [16]:
import sqlite3

conn = sqlite3.connect('/Users/valen/Desktop/SAS/BC.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM NN")
rows = cursor.fetchall()

dataNN = [{'id': row[0], 'rdm_float': row[1]} for row in rows]

conn.close()

In [17]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['Todo']
collection = db['CRUD_exercise']

collection.insert_many(dataNN)

print(collection.name)

Data inserted into MongoDB collection: CRUD_exercise


**TODO 4**: Get documents where authors key exist in the collection "CRUD_exercise".

In [18]:
documents = collection.find({'authors': {'$exists': True}})

for doc in documents:
    print(doc)


**TODO 5**: Change the documents where x = 4 to x = 1.

In [21]:
collection.update_many({'x': 4}, {'$set': {'x': 1}})


UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)


**TODO 6**: Find documents where author is not_mike and set author as real_mike.

In [22]:
collection.update_many({'author': 'not_mike'}, {'$set': {'author': 'real_mike'}})

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)


**TODO 7**: Delete documents where author is real_mike.

In [23]:
collection.delete_many({'author': 'real_mike'})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

#### Managing DB

**TODO 8**: create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

In [38]:
import time
import numpy as np
start_time_with_index = time.time()
doc = [{'id': i, 'values': np.random.rand(10).tolist()} for i in range(0, 1000000, 2)]

insert_result = collection.insert_many(doc)
end_time_no_index = time.time()
print(f"Inserted {len(insert_result.inserted_ids)} doc.")

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq\\backend\\cython\\checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


Inserted 500000 doc.


In [39]:
import pymongo
start_time_with_index = time.time()
collection.create_index([('id', pymongo.ASCENDING)])
end_time_with_index = time.time()

In [40]:
print(f"Sans index: {end_time_no_index - start_time_no_index} seconds.")
print(f"Avec index: {end_time_with_index - start_time_with_index} seconds.")

Sans index: 101.88962388038635 seconds.
Avec index: 0.0010027885437011719 seconds.


Oui, nous remarquons que la requête sans index mets beaucoup plus de temps que la requête avec index

**TODO 9**: create a random collection in a random db and put the new collection in the tutorial DB

In [41]:
import pymongo
import numpy as np
from pymongo import MongoClient
import string
import random

def random_string(length=10):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

client = MongoClient('localhost', 27017)

random_db_name = random_string()
random_col_name = random_string()

random_db = client[random_db_name]
random_collection = random_db[random_col_name]

documents = [{'id': i, 'values': np.random.rand(10).tolist()} for i in range(10)]
random_collection.insert_many(documents)
print(f"Data inserted into {random_db_name}.{random_col_name}")

tutorial_db = client['tutorial']
tutorial_collection = tutorial_db[random_col_name]

data_to_transfer = list(random_collection.find({}))
tutorial_collection.insert_many(data_to_transfer)
print(f"Data copied to tutorial.{random_col_name}")

client.drop_database(random_db_name)
print(f"Database {random_db_name} deleted")

print("Collections in 'tutorial' database:", tutorial_db.list_collection_names())


Data inserted into zwroewvwme.erqrbfzety
Data copied to tutorial.erqrbfzety
Database zwroewvwme deleted
Collections in 'tutorial' database: ['image', 'benchmark_2', 'example', 'benchmark', 'arxiv_api', 'erqrbfzety', 'example_to_dump']


**TODO 10**: What is the difference between an inner join and an outer join ? Is the query seen during course an inner or outer join ? Play with the query to show all the joins.

#### Real world problems

**TODO 11**:  Use the oaipmh and api code get papers after January 2020 and for "cs,math,econ" categories. Insert them in MongoDB. Import only the first 200. How is it sorted ? How can you define your own sort()? Query papers to get papers after 2021, which have 3 authors and with domain "cs".

**TODO 12**: Do the same as TODO 8 but with the connection to the cluster. Then check the metrics and take screenshot of opcounters, logical size and connections.

**TODO 13**: Download a random image and store it in a collection.

**TODO 14**: Try to store a pandas dataframe in mongoDB (array with rownames, array with colnames and matrix with values)

**TODO 15**: Insert the movie_review.tsv data into mongodb. Then query it to find the number of review that are positive and negative review. Fetch the docs which have "unexpected" in their review, how many are they ? Think of a clever way to count the number of words in the review using MongoDB (hint: Transform the review text before the insert in MongoDB) and create a density of number of words per review.

**TODO 16**: Download a [sound sample](https://freesound.org/browse/). Try to store it in MongoDB 

**TODO 17**: Create a collection with 30M observation with a single key : "year" which is a random value between 2000-2020. Get documents with year = 2000. Does using an index helps ? 